In [1]:
from PIL import Image
import requests
import matplotlib.pyplot as plt
import numpy as np
import scipy
import pandas as pd
import os
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from tqdm import tqdm
from transformers import CLIPProcessor, CLIPModel
from transformers import logging
logging.set_verbosity_error()

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
# model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
# processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")
# model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
# processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [3]:
from transformers import CLIPConfig, CLIPTextConfig, CLIPVisionConfig

config_text = CLIPTextConfig(max_position_embeddings=512)
config_vision = CLIPVisionConfig()

config = CLIPConfig.from_text_vision_configs(config_text, config_vision)
config.name_or_path = "openai/clip-vit-base-patch32"

model = CLIPModel(config)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [4]:
print(model.num_parameters()) # 427616513 params for large-patch14
                              # 151277313 params for base-patch32
                              # 149620737 params for base-patch16

151500033


In [5]:
# # boilerplate code from huggingface docs
# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw)
# plt.imshow(image)
# inputs = processor(text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True)

# outputs = model(**inputs)
# logits_per_image = outputs.logits_per_image # this is the image-text similarity score
# probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities
# print(outputs.text_embeds.shape)

In [6]:
# Loading data
caption_path = "./summaries"
cartoon_path = "./cartoons"
ids = set()
cap_paths = []
img_paths = []
for file in os.listdir(caption_path):
    file_path = os.path.join(caption_path, file)
    if not os.path.isfile(file_path) or file[-3:]!='csv' or file[:3] in ids:
        continue
    ids.add(file[:3])
    cap_paths.append(file_path)
for file in os.listdir(cartoon_path):
    file_path = os.path.join(cartoon_path, file)
    if not os.path.isfile(file_path) or file[-3:]!='jpg':
        continue
    if file[:3] not in ids: 
        print("extra_img: ", file)
        continue
    img_paths.append(file_path)
id_list = list(ids)
id_list.sort()
cap_paths.sort()
img_paths.sort()
assert(len(img_paths) == len(cap_paths))

extra_img:  890.jpg


In [7]:
# pca = PCA(n_components=50)
# condensed_data = pca.fit_transform(img1_cap)
# print(condensed_data.shape, np.max(condensed_data), np.min(condensed_data))
# tsne = TSNE()
# tsne_embeds = tsne.fit_transform(condensed_data)
# norm_tsne_embeds = np.zeros(tsne_embeds.shape)
# mins = np.min(tsne_embeds, axis=0, keepdims=True)
# maxes = np.max(tsne_embeds, axis=0, keepdims=True)
# norm_tsne_embeds = (tsne_embeds-mins)/(maxes-mins)
# print(norm_tsne_embeds.shape, np.max(norm_tsne_embeds), np.min(norm_tsne_embeds))
# print(norm_tsne_embeds[:5])

# TSNE Processing Functions

In [8]:
def get_embeds(model, image, caption_list):
    inputs = processor(text=caption_list, images=image, return_tensors="pt", padding=True)
    outputs = model(**inputs)
    text_embeds = outputs.text_embeds.detach().numpy()
    img_embeds = outputs.image_embeds.detach().numpy()
    return text_embeds, img_embeds

In [9]:
pca = PCA(n_components=50)
tsne = TSNE()
def get_tsne_embeds(img_df):
    caps = img_df["cap_feat"]
    cap_data = np.vstack(caps)
    condensed_data = pca.fit_transform(cap_data)
    tsne_embeds = tsne.fit_transform(condensed_data)
    norm_tsne_embeds = np.zeros(tsne_embeds.shape)
    mins = np.min(tsne_embeds, axis=0, keepdims=True)
    maxes = np.max(tsne_embeds, axis=0, keepdims=True)
    norm_tsne_embeds = (tsne_embeds-mins)/(maxes-mins)
    norm_tsne_embeds_df = pd.DataFrame({'X': norm_tsne_embeds[:,0], 'Y': norm_tsne_embeds[:,1], "caption": caps})
    return norm_tsne_embeds_df

In [10]:
def filter_long_captions(cap_df, filter_len=50):
    cap_df['sentence_len'] = cap_df['caption'].apply(lambda x: len(x.split()))
    return cap_df[cap_df['sentence_len'] <= filter_len].reset_index()

In [11]:
batch_size = 32
def get_embeds_from_path(img_id, img_path, cap_path, batch_size=32, filter_len=100):
    df_list = []
    img = Image.open(img_path)
    inputs = processor(text=['test'], images=img, return_tensors="pt", padding=True)
    img_feat = model(**inputs).image_embeds.detach().numpy()
    cap_csv = pd.read_csv(cap_path)
    cap_csv = filter_long_captions(cap_csv, filter_len)
    cap_csv["img_id"] = img_id
    cap_csv["cap_feat"] = None
    cap_csv["img_feat"] = [img_feat]*len(cap_csv.index)
    caption_list = []
    for idx,row in cap_csv.iterrows():
        caption_list.append(row['caption'])
        if (idx == (len(cap_csv.index)-1)):
            text_embeds, _ = get_embeds(model, img, caption_list)
            for i,embed in enumerate(text_embeds):
                cap_csv.at[idx-len(text_embeds)+1+i,"cap_feat"] = text_embeds[i]
            caption_list = []
        if (len(caption_list) != batch_size):
            continue
        text_embeds, _ = get_embeds(model, img, caption_list)
        for i,embed in enumerate(text_embeds):
            cap_csv.at[idx-batch_size+1+i,"cap_feat"] = text_embeds[i]
        caption_list = []
    df_list.append(cap_csv)
    return pd.concat(df_list)

In [12]:
# relevant_columns = ["caption", "img_id", "mean","votes"]
# def process_dataset_tsne(id_list, img_paths, cap_paths):
#     tsne_list = []
#     cap_list = []
#     for img_id, img_path, cap_path in zip(id_list, img_paths, cap_paths):
#         # process for TSNE data

#         img_df = get_embeds_from_path(img_id, img_path, cap_path)
#         tsne_feats = get_tsne_embeds(img_df)
#         tsne_feats["img_id"] = img_id
#         tsne_list.append(tsne_feats)

#         # save complete dataset
#         cap_csv = pd.read_csv(cap_path)
#         cap_csv["img_id"] = img_id
#         # print(list(cap_csv.columns))
#         cap_list.append(cap_csv[relevant_columns])
#         # break

#     return pd.concat(tsne_list), pd.concat(cap_list)

# Create complete TSNE DataFrame

In [13]:
relevant_columns = ['rank', 'caption', 'mean', 'precision', 'votes', 'not_funny', 'somewhat_funny', 'funny', 'img_id', 'cap_feat', 'img_feat']
def process_dataset(id_list, img_paths, cap_paths):
    ################################################################
    # Processes all data to get the relevant columns and tsne data #
    ################################################################
    cap_folder = 'caption_data'  # Folder to save cap_df files
    tsne_folder = 'tsne_data'  # Folder to save tsne_df files
    
    # Ensure the folders exist
    os.makedirs(cap_folder, exist_ok=True)
    os.makedirs(tsne_folder, exist_ok=True)
    
    cap_failed_ids = []
    tsne_failed_ids = []
    
    for img_id, img_path, cap_path in tqdm(zip(id_list, img_paths, cap_paths), total=len(id_list)):
        # Define file paths
        cap_file_path = os.path.join(cap_folder, f'cap_df_{img_id}.pkl')
        tsne_file_path = os.path.join(tsne_folder, f'tsne_df_{img_id}.pkl')
        
        if (not os.path.exists(cap_file_path)) and (not os.path.exists(tsne_file_path)):
            print("Running for image id:", img_id)
            try:
                cap_df = get_embeds_from_path(img_id, img_path, cap_path)
                cap_df.to_pickle(cap_file_path)
                try:
                    tsne_df = get_tsne_embeds(cap_df)
                    tsne_df["img_id"] = img_id
                    tsne_df.to_pickle(tsne_file_path)
                    print(f'Saved: {cap_file_path} and {tsne_file_path}')
                except:
                    tsne_failed_ids.append(img_id)
                
            except:
                cap_failed_ids.append(img_id)
    print('cap_failed_ids:', cap_failed_ids)
    print('tsne_failed_ids:', tsne_failed_ids)
    print('Equal:', tsne_failed_ids == cap_failed_ids)
    return cap_failed_ids, tsne_failed_ids

In [14]:
process_dataset(id_list, img_paths, cap_paths)

  0%|          | 0/379 [00:00<?, ?it/s]

Running for image id: 661


 40%|███▉      | 151/379 [00:00<00:00, 784.86it/s]

Running for image id: 662


 40%|████      | 152/379 [01:16<02:42,  1.40it/s] 

...

100%|█████████▉| 378/379 [4:13:11<01:14, 74.06s/it]

Saved: caption_data\cap_df_888.pkl and tsne_data\tsne_df_888.pkl
Running for image id: 889


100%|██████████| 379/379 [4:14:54<00:00, 40.35s/it]

Saved: caption_data\cap_df_889.pkl and tsne_data\tsne_df_889.pkl
cap_failed_ids: ['661', '669', '670', '671', '675', '691', '692', '693', '694', '695', '696', '698', '702', '704', '707', '709', '711', '712', '713', '715', '716', '721', '731', '748', '752', '760', '769', '771', '772', '782', '793', '814', '815', '818', '819', '826', '828', '835', '849', '863', '871', '874', '878', '879']
tsne_failed_ids: []
Equal: False


(['661',
  '669',
  '670',
  '671',
  '675',
  '691',
  '692',
  '693',
  '694',
  '695',
  '696',
  '698',
  '702',
  '704',
  '707',
  '709',
  '711',
  '712',
  '713',
  '715',
  '716',
  '721',
  '731',
  '748',
  '752',
  '760',
  '769',
  '771',
  '772',
  '782',
  '793',
  '814',
  '815',
  '818',
  '819',
  '826',
  '828',
  '835',
  '849',
  '863',
  '871',
  '874',
  '878',
  '879'],
 [])

In [15]:
# tsne_df, condensed_cap_df = process_dataset_tsne(id_list, img_paths, cap_paths)
# tsne_df.head(5)

In terms of preprocessing, we will likely want to save the original dataset dataframes without the precison or score breakdown columns. The TSNE dataframe above can be saved as is since it should be possible to filter for the correct img_id when necessary. 

In [16]:
# condensed_cap_df.head(5)

# Feed Forward Network